# Мини-проект

## ПОСТАНОВКА ЗАДАЧИ

К вам попали результаты A/A/B-тестирования от одного известного маркетплейса. 

sample_a, sample_c — АА-группы, sample_b — отдельная группа. 

В каждом датасете есть три типа действий пользователей: 0 — клик, 1 — просмотр и 2 — покупка. 

Маркетплейс ориентируется на следующие метрики:

* ctr (отношение кликов к просмотрам товаров);
* purchase rate (отношение покупок к просмотрам товаров);
* gmv (оборот, сумма произведений количества покупок на стоимость покупки), где считаем 1 сессию за 1 точку (1 сессия на 1 пользователя).

Данные уже почищены по сессиям, вы можете использовать их в агрегированном виде. Ваша задача — понять, нет ли проблемы с разъезжанием сплитов и улучшает ли алгоритм B работу маркетплейса.

Тест Шапиро-Уилка проведите на alpha=0.01.

In [84]:
import numpy as np
import pandas as pd

In [85]:
sample_a = pd.read_csv('sample_a.csv')
sample_b = pd.read_csv('sample_b.csv')
sample_c = pd.read_csv('sample_c.csv')
prices = pd.read_csv('item_prices.csv')

In [86]:
sample_a.sort_values(['user_id', 'item_id']).head()

,user_id,item_id,action_id
1158951,241,21,1
800805,241,36,1
826737,241,39,1
102557,241,50,1
17980,241,64,1


In [87]:
sample_a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1188912 entries, 0 to 1188911
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1188912 non-null  int64
 1   item_id    1188912 non-null  int64
 2   action_id  1188912 non-null  int64
dtypes: int64(3)
memory usage: 27.2 MB


In [88]:
sample_b.head()

,user_id,item_id,action_id
0,118375,4105,1
1,107569,8204,1
2,175990,880,1
3,160582,9568,0
4,123400,4000,1


In [89]:
sample_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1198438 entries, 0 to 1198437
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1198438 non-null  int64
 1   item_id    1198438 non-null  int64
 2   action_id  1198438 non-null  int64
dtypes: int64(3)
memory usage: 27.4 MB


In [90]:
sample_c.head()

,user_id,item_id,action_id
0,274623,2863,1
1,265472,343,1
2,242779,6009,0
3,275009,2184,1
4,268104,3134,2


In [91]:
sample_c.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1205510 entries, 0 to 1205509
Data columns (total 3 columns):
 #   Column     Non-Null Count    Dtype
---  ------     --------------    -----
 0   user_id    1205510 non-null  int64
 1   item_id    1205510 non-null  int64
 2   action_id  1205510 non-null  int64
dtypes: int64(3)
memory usage: 27.6 MB


In [92]:
prices.head()

,item_id,item_price
0,338,1501
1,74,647
2,7696,825
3,866,875
4,5876,804


In [93]:
prices[prices.item_id == 338]

,item_id,item_price
0,338,1501
797,338,1151
977,338,1047


In [94]:
# Проведена проверка, что нет ситуаций,
# когда происходит покупка/клик без действия просмотра.
set_a = pd.get_dummies(sample_a, prefix=['action_id'], columns=['action_id'])
set_a_gr = set_a.groupby(['user_id', 'item_id']).sum()
set_a_gr[set_a_gr['action_id_1']==0]

,,action_id_0,action_id_1,action_id_2
user_id,item_id,,,


In [95]:
set_b = pd.get_dummies(sample_b, prefix=['action_id'], columns=['action_id'])
set_b_gr = set_b.groupby(['user_id', 'item_id']).sum()
set_b_gr[set_b_gr['action_id_1']==0]

,,action_id_0,action_id_1,action_id_2
user_id,item_id,,,


In [96]:
set_c = pd.get_dummies(sample_c, prefix=['action_id'], columns=['action_id'])
set_c_gr = set_c.groupby(['user_id', 'item_id']).sum()
set_c_gr[set_c_gr['action_id_1']==0]

,,action_id_0,action_id_1,action_id_2
user_id,item_id,,,


In [97]:
# Удалены дубли.
sample_a.duplicated().value_counts()

False    1188912
dtype: int64

In [98]:
sample_b.duplicated().value_counts()

False    1198438
dtype: int64

In [99]:
sample_c.duplicated().value_counts()

False    1205510
dtype: int64

In [100]:
prices.duplicated(subset=['item_id']).value_counts()

False    955
True      45
dtype: int64

In [101]:
prices.drop_duplicates(subset=['item_id'], keep='last', inplace=True)

In [102]:
# The Shapiro-Wilk test is a test of normality.
# It is used to determine whether or not
# a sample comes from a normal distribution.
from scipy.stats import shapiro

In [103]:
shapiro(set_a_gr.values)

/Users/user/opt/anaconda3/lib/python3.8/site-packages/scipy/stats/morestats.py:1760: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.6383861899375916, pvalue=0.0)

In [104]:
shapiro(set_b_gr.values)

ShapiroResult(statistic=0.6361701488494873, pvalue=0.0)

In [105]:
shapiro(set_c_gr.values)

ShapiroResult(statistic=0.6362341046333313, pvalue=0.0)

In [108]:
# Рассчитаны метрики по датасетам,

a_merged = pd.merge(sample_a, prices, on=['item_id'])
ctr_a = a_merged.action_id.value_counts()[0] / a_merged.action_id.value_counts()[1]
pr_a = a_merged.action_id.value_counts()[2] / a_merged.action_id.value_counts()[1]
gvm_a = a_merged[a_merged.action_id==2]['item_price'].sum()

print('CTR A = ', ctr_a)
print('PR_A = ', pr_a)
print('GVM_A = ', gvm_a)
print()

b_merged = pd.merge(sample_b, prices, on=['item_id'])
ctr_b = b_merged.action_id.value_counts()[0] / b_merged.action_id.value_counts()[1]
pr_b = b_merged.action_id.value_counts()[2] / b_merged.action_id.value_counts()[1]
gvm_b = b_merged[b_merged.action_id==2]['item_price'].sum()

print('CTR A = ', ctr_b)
print('PR_A = ', pr_b)
print('GVM_A = ', gvm_b)
print()

c_merged = pd.merge(sample_c, prices, on=['item_id'])
ctr_c = c_merged.action_id.value_counts()[0] / c_merged.action_id.value_counts()[1]
pr_c = c_merged.action_id.value_counts()[2] / c_merged.action_id.value_counts()[1]
gvm_c = c_merged[c_merged.action_id==2]['item_price'].sum()

print('CTR A = ', ctr_c)
print('PR_A = ', pr_c)
print('GVM_A = ', gvm_c)
print()


CTR A =  0.2
PR_A =  0.049999474309505534
GVM_A =  51183991

CTR A =  0.16000046260228504
PR_A =  0.09999989486311704
GVM_A =  102261317

CTR A =  0.20999956806686745
PR_A =  0.059999726091184244
GVM_A =  61236564



In [114]:
# проведено общее сравнение метрик.
results = pd.DataFrame([[ctr_a, pr_a, gvm_a/1000000],
                        [ctr_b, pr_b, gvm_b/1000000],
                        [ctr_c, pr_c, gvm_c/1000000]], index=['A', 'B', 'C'],
                      columns=['CTR', 'PR', 'GVM'])

results

,CTR,PR,GVM
A,0.20,0.049999,51.183991
B,0.16,0.100000,102.261317
C,0.21,0.060000,61.236564


sample_b (отдельная группа) в целом отличается по метрикам от АА-групп (A, C). Так, при уменьшающейся CTR (0.2 --> 0.16) мы видим почти двукратный прирост purchase rate (0.05-0.6 --> 0.1) и увеличенный GVM (50-60m --> 102m)

In [121]:
# Проведён тест равенства долей для A и C групп по всем метрикам.

# Сделан вывод.

import scipy as sp
from statsmodels.stats.proportion import proportions_ztest

print(proportions_ztest(
    count=[a_merged.action_id.value_counts()[0],
           c_merged.action_id.value_counts()[0]], 
    nobs=[a_merged.action_id.value_counts()[1],
          c_merged.action_id.value_counts()[1]])[1])

print(proportions_ztest(
    count=[a_merged.action_id.value_counts()[2],
           c_merged.action_id.value_counts()[2]], 
    nobs=[a_merged.action_id.value_counts()[1],
          c_merged.action_id.value_counts()[1]])[1])

print(sp.stats.ttest_ind(a_merged.item_price.values, c_merged.item_price.values))

2.354942111153043e-65
8.035152879055818e-201
Ttest_indResult(statistic=-0.2776446866113944, pvalue=0.781285149246036)


In [122]:
# Проведён тест равенства долей для A и B групп по всем метрикам.

# Сделан вывод


print(proportions_ztest(
    count=[a_merged.action_id.value_counts()[0],
           b_merged.action_id.value_counts()[0]], 
    nobs=[a_merged.action_id.value_counts()[1],
          b_merged.action_id.value_counts()[1]])[1])

print(proportions_ztest(
    count=[a_merged.action_id.value_counts()[2],
           b_merged.action_id.value_counts()[2]], 
    nobs=[a_merged.action_id.value_counts()[1],
          b_merged.action_id.value_counts()[1]])[1])

print(sp.stats.ttest_ind(a_merged.item_price.values, b_merged.item_price.values))

0.0
0.0
Ttest_indResult(statistic=-0.8802732558964812, pvalue=0.37871138560806505)


Имеются различия в метриках CTR и PR между выборками A и B, но не A и C.

Имеются различия в метрике GVM как между выборками A и B, так и A и C.